# Nombre: Arturo Lazcano
## RUT: 20.470.051-6

# CC3001 Primavera 2020 Tarea 5
# Inserción en la raíz en un ABB
## Profesores
Sección 1 Benjamin Bustos • Sección 2 Jérémy Barbay • Sección 3 Patricio Poblete / Nelson Baloian

---

# CONTEXTO

El algoritmo usual de inserción en un ABB realiza una búsqueda infurctuosa para la nueva llave, y la coloca en el lugar de la hoja en donde termina la búsqueda. Se dice que es una *inserción en las hojas*.

Por contraposición, un algoritmo de *inserción en la raíz* deja la nueva llave como raíz del árbol resultante. En el apunte aparece un algoritmo de este tipo basado en *rotaciones*.

El objetivo de esta tarea es implementar inserción en la raíz de un ABB usando un algoritmo alternativo, basado en la operación ``cut``(corte).

Para comenzar, recordemos esta implementación del algoritmo de inserción usual (inserción en las hojas) adaptada del apunte:

In [4]:
class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
        
    def insert(self,x):
        assert x!=self.info
        if x<self.info:
            return Nodoi(self.izq.insert(x),self.info,self.der)
        else:
            return Nodoi(self.izq,self.info,self.der.insert(x))
        
    def __str__(self):
        return "("+self.izq.__str__()+str(self.info)+self.der.__str__()+")"

class Nodoe:
    def __init__(self):
        pass
    
    def insert(self,x):
        return Nodoi(Nodoe(),x,Nodoe())
    
    def __str__(self):
        return"☐"

class Arbol:
    def __init__(self,raiz=Nodoe()):
        self.raiz=raiz     
        
    def insert(self,x):
        self.raiz=self.raiz.insert(x)
    
    def __str__(self):        
        return self.raiz.__str__()

Probemos este algoritmo con una secuencia de inserciones:

In [5]:
lista=[4,9,6,2,7,1,8,3,5]
a=Arbol()
for x in lista:
    a.insert(x)
print(a)

(((☐1☐)2(☐3☐))4(((☐5☐)6(☐7(☐8☐)))9☐))


Para definir el algoritmo de inserción en la raíz, definimos primero la operación ``cut(x)``. La idea es la siguiente: dado un árbol $A$ y una llave $x$ que no está en el árbol, el resultado de ejecutar $\text{cut}(x)$ sobre el árbol $A$ es una tupla $(A_{<x},A_{>x})$, que contiene como primera componente el árbol que contiene solo las llaves que son menores que $x$, y similarmente la segunda componente es el árbol con las llaves mayores que $x$. Por ejemplo:

![T5-cut](https://github.com/ppoblete/CC3001-2020-2-Tareas/blob/master/T5-cut.png?raw=1)

La operación ``cut(x)`` se puede definir recursivamente:

Para el caso de un árbol no vacío con raíz $r$, con $x<r$:

![T5-cut2](https://github.com/ppoblete/CC3001-2020-2-Tareas/blob/master/T5-cut2.png?raw=1)

El caso $x>r$ es simétrico.

El caso de borde es que al hacer ``cut(x)``sobre un nodo ☐, el resultado es una tupla (☐,☐).

En base a esta operación ``cut(x)``, es fácil definir una operación ``root_insert(x)``. En efecto, para insertar una nueva llave $x$ en la raíz de un árbol $A$, se hace un corte con $x$ al árbol $A$, y luego se construye el árbol

![T5-cut3](https://github.com/ppoblete/CC3001-2020-2-Tareas/blob/master/T5-cut3.png?raw=1)

# INSTRUCCIONES

## Parte 1: ``cut(x)`` y ``root_insert(x)``

Modifique las clases provistas para agregar la operación ``cut(x)``y la operación ``root_insert(x)``. Escriba acá su código para las clases completas:

In [11]:
class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
        
    def insert(self,x):
        assert x!=self.info
        if x<self.info:
            return Nodoi(self.izq.insert(x),self.info,self.der)
        else:
            return Nodoi(self.izq,self.info,self.der.insert(x))
    
    def root_insert(self,izq,x,der):
        assert x!=self.info
        return Nodoi(izq,x,der)
        
    
    def __str__(self):
        return "("+self.izq.__str__()+str(self.info)+self.der.__str__()+")"
    

class Nodoe:
    def __init__(self):
        self.info="()"
        self.der=()
        self.izq=()
    
    def insert(self,x):
        return Nodoi(Nodoe(),x,Nodoe())
    
    def root_insert(self,izq,der):
        return Nodoi()
    
    def __str__(self):
        return"☐"
    
    def root_insert(self,x):
        return Nodoi(Nodoe())

class Arbol:
    def __init__(self,raiz=Nodoe()):
        self.raiz=raiz
        
    def insert(self,x):
        self.raiz=self.raiz.insert(x)
    
    def __str__(self):        
        return self.raiz.__str__()
    
    def recorrer(self):
        recorrido=[]
        def aux(r):
            if r.info=="()":
                return
            recorrido.append(r.info)
            aux(r.izq)
            aux(r.der)
        aux(self.raiz)
        borrar=[]
        for i in range(0,len(recorrido)):
            if recorrido[i]==():
                borrar.append(i)
        borrar.reverse()
        for k in borrar:
            recorrido.pop(k)
        return recorrido
    
    def cut(self,x):
        r=self.recorrer()
        subArboles=([],[])
        for i in r:
            if i<x:
                subArboles[0].append(i)
            elif i>x:
                subArboles[1].append(i)
            else:
                print("valor x ya existe en el arbol")
        aizq=Arbol()
        ader=Arbol()
        for i in subArboles[0]:
            aizq.insert(i)
        for d in subArboles[1]:
            ader.insert(d)
        return (aizq,ader)
    
    def root_insert(self,x):
        if self.raiz.info=="()":
            self.raiz=Nodoi(Nodoe(),x,Nodoe())            
        else:
            SA=self.cut(x)
            self.raiz=Nodoi(SA[0].raiz,x,SA[1].raiz)

## Parte 2: Evaluación experimental

A continuación, ejecute lo siguiente para probar su implementación:

In [14]:
lista=[4,9,6,2,7,1,8,3,5]
a=Arbol()
for x in reversed(lista):
    a.root_insert(x)
print("resultado nuevo = ",a)

lista=[4,9,6,2,7,1,8,3,5]
a=Arbol()
for x in lista:
    a.insert(x)
print("resultado antiguo=",a)

resultado nuevo =  (((☐1☐)2(☐3☐))4(((☐5☐)6(☐7(☐8☐)))9☐))
resultado antiguo= (((☐1☐)2(☐3☐))4(((☐5☐)6(☐7(☐8☐)))9☐))


Si todo está bien, el resultado de insertar al lista reversa usando ``root_insert``debería ser igual al obtenido insertando la lista original usando ``insert``.

## ¿Qué hay que entregar?

Usted debe entregar este mismo archivo, modificado de acuerdo a lo que se pide. Haga todos los cambios necesarios para explicar y documentar adecuadamente su código. No olvide poner su nombre.